## Define the problem

In [173]:
using Convex;
using MathOptInterface;
const MOI = MathOptInterface;

In [146]:
m, n = 3, 4
x = Variable((n , 1) )

Variable
size: (4, 1)
sign: real
vexity: affine
id: 709…285

In [147]:
# defining params in julia

F = Variable((m, n))
g = Variable((m, 1))
lambd = Variable((1, 1), Positive())

F.value = 10*ones(m, n)
g.value = 10*ones(m, 1)
lambd.value = 10*ones(1, 1)

fix!(F)
fix!(g)
fix!(lambd)

Variable
size: (1, 1)
sign: positive
vexity: constant
id: 165…355
value: [10.0]

In [148]:
objective_fn = norm(F * x - g) + lambd*norm(x)
constraints = [x >= 0]
problem = minimize(objective_fn, constraints)

minimize
└─ + (convex; positive)
   ├─ norm2 (convex; positive)
   │  └─ + (affine; real)
   │     ├─ …
   │     └─ …
   └─ * (convex; positive)
      ├─ positive variable (fixed) (id: 165…355)
      └─ norm2 (convex; positive)
         └─ …
subject to
└─ >= constraint (affine)
   ├─ 4-element real variable (id: 709…285)
   └─ 0

status: `solve!` not called yet

## Get cone problem data

In [165]:
problem.constraints[1]

>= constraint (affine)
├─ 4-element real variable (id: 709…285)
└─ 0

In [162]:
unique_conic_forms = Convex.UniqueConicForms()
objective, objective_var_id = Convex.conic_form!(problem, unique_conic_forms)

(Convex.ConicObj(OrderedCollections.OrderedDict{UInt64,Tuple{Union{Number, AbstractArray},Union{Number, AbstractArray}}}(0xcbe08e9204143a16=>(
  [1, 1]  =  1.0, 1×1 SparseMatrixCSC{Float64,Int64} with 0 stored entries),0xda59128f4ac0b6bc=>(1×1 SparseMatrixCSC{Float64,Int64} with 0 stored entries, 1×1 SparseMatrixCSC{Float64,Int64} with 0 stored entries))), 0xcbe08e9204143a16)

In [157]:
objective

Convex.ConicObj(OrderedCollections.OrderedDict{UInt64,Tuple{Union{Number, AbstractArray},Union{Number, AbstractArray}}}(0x1fca946963694d45=>(
  [1, 1]  =  1.0, 1×1 SparseMatrixCSC{Float64,Int64} with 0 stored entries),0xda59128f4ac0b6bc=>(1×1 SparseMatrixCSC{Float64,Int64} with 0 stored entries, 1×1 SparseMatrixCSC{Float64,Int64} with 0 stored entries)))

In [170]:
for con in unique_conic_forms.constr_list
    for obj in con.objs
        for (k,v) in obj.mapping
            print(Array(v[1]))
            print('\n')
        end
    end
    print('\n')
end

[1.0]
[0.0]
[10.0 10.0 10.0 10.0; 10.0 10.0 10.0 10.0; 10.0 10.0 10.0 10.0]
[-10.0; -10.0; -10.0]

[1.0]
[0.0]
[1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]
[0.0; 0.0; 0.0; 0.0]

[1.0]
[0.0]
[10.0]
[-1.0]

[1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]
[0.0; 0.0; 0.0; 0.0]



In [176]:
conic_constraints = unique_conic_forms.constr_list
conic_constr_to_constr = unique_conic_forms.conic_constr_to_constr
id_to_variables = unique_conic_forms.id_to_variables

MOI_constr_fn = Union{MOI.VectorAffineFunction{Float64},MOI.SingleVariable}[]
MOI_sets = Any[]
for conic_constr in conic_constraints
    set, constr_fn = Convex.make_MOI_constr(conic_constr, var_to_indices, id_to_variables, Float64)
    push!(MOI_sets, set)
    push!(MOI_constr_fn, constr_fn)
end

UndefVarError: UndefVarError: var_to_indices not defined